# Part 1. Select and download image data

#### Package imports

In [ ]:
#!jupyter nbextension enable --py widgetsnbextension

In [ ]:
#note: if extension not working try running the following from the terminal: jupyter dashboards quick-setup --user

In [ ]:
#!jupyter nbextension enable jupyter_dashboards --user --py

In [1]:
!pip install cufflinks

    100% |████████████████████████████████| 51kB 1.4MB/s ta 0:00:01
  Running setup.py bdist_wheel for cufflinks ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/1d/79/29/bb076c9ab0bd7a932c33e7b6c7eb5d00d84001076f16549ec0
  Running setup.py bdist_wheel for colorlover ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/b8/b0/18/76d3f3088cc73950ed1aa44ea074b93ed37309fea57dec78dc
Successfully built cufflinks colorlover


In [2]:
!pip install utm

  Running setup.py bdist_wheel for utm ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/18/f5/d5/7ed9ab14cc72ab038bdd0eab3c16476030cd17d692f2468b1a
Successfully built utm


In [3]:

#interactive web map package
import folium 
from folium import features
import shapefile
from json import dumps
from folium import plugins

#Miscellanous
from IPython.core.display import display #display inline package
from ipywidgets import interact, interactive, fixed, FloatProgress
import ipywidgets as widgets
import pdb #debugging tool
import pandas as pd # pandas dataframe package
import pandas
from datetime import datetime #date conversion tool
from xlrd.xldate import xldate_as_tuple #xldate converter
from urllib2 import urlopen #get data from web tool
#from collections import OrderedDict
#import warnings
#warnings.filterwarnings('ignore')
from IPython.display import Image
import utm #lat long to utms
import zipfile #unzip tool
from IPython.display import clear_output
from scipy.signal import correlate #rainfall correlation

#numpy packages and tools
import numpy as np
from numpy import linspace

import rasterio #io raster data 
#from osgeo import gdal

#Plotly tools
import plotly.tools as tls
tls.set_credentials_file(username='NTPlotly', api_key='48kd2al3f2') #my plotly credentials (please dont use they cost me $$)
import plotly.plotly as py
#from plotly.graph_objs import *
import plotly.graph_objs as go
from plotly import tools
import cufflinks as cf
import matplotlib

#### Data import

##### Import site coords

Import a list of the ~500 sites across the study area with each sites latitude and longitude.

In [4]:
#Function to get a number of lists and shapefiles about the study area.

def getdata():
    
    
        #reads in a prepared list of sites asat September 2016 from dropbox
        url = "https://dl.dropboxusercontent.com/u/53285700/csvs/siteDetails.csv"  # dl=1 is important
        u = urlopen(url)
        data = u.read()
        u.close()
        filename = "siteDetails.csv"

        with open(filename, "wb") as f :
            f.write(data)

        df = pd.read_csv(filename) 

        #reads in a prepared list of images for downloading, note the images have been created, stored and extracted into dropbox folder from the dsiti system using the following script on the hpc: UQ_createNPVChips.py
        url = "https://dl.dropboxusercontent.com/u/53285700/images/images.txt"  # dl=1 is important
        u = urlopen(url)
        data = u.read()
        u.close()
        filename = "imageNames.csv"

        with open(filename, "wb") as f :
            f.write(data)

        images = pd.read_csv(filename,header=None)

        #read in zip file containing shapefile of sub ibra bioregions
        url = "https://www.dropbox.com/s/d63c5m0jac6jvct/NTSubIbra.zip?dl=1"  # dl=1 is important
        u = urlopen(url)
        data = u.read()
        u.close()
        filename = "NTSubIbra.zip"

        with open(filename, "wb") as f :
            f.write(data)
            
        return df, images

In [5]:
#call the getdata function - inline
df,images = getdata()

HTTPError: HTTP Error 404: Not Found

#### Step 1. Click button below to create a web map with field sites

In [4]:
#display map and select site (manual at this stage, plan is to use click to fire download)

def onMapButtonClick(b):

    '''
    #unzip shapefile of bioregions
    zip1 = zipfile.ZipFile('data/NTSubIbra.zip')
    zip1.extractall()
    zip1.close()
    
    #Convert IBRA shapefile to geojsn

    #Read in the bioregion shp
    reader = shapefile.Reader("data/NTSubIbra.shp")
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]
    buffer1 = []
    for sr in reader.shapeRecords():
        atr = dict(zip(field_names, sr.record))
        geom = sr.shape.__geo_interface__
        buffer1.append(dict(type="Feature", \
        geometry=geom, properties=atr))

    #convert to geojs
    geojson = open("data/NTSubIbra.json", "w")
    geojson.write(dumps({"type": "FeatureCollection","features": buffer1}, indent=2) + "\n")
    geojson.close()
    '''
    print "Please be patient as this can take up to 30 secs"
    #create the map centered on Alice Springs with a DEM background
    m = folium.Map(location=[-23.7003, 133.8808],zoom_start=7) # create the web map from a basic terrain and street map
    
    #pdb.set_trace()
    

    url = 'http://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/MapServer/tile/{z}/{y}/{x}'

    m.add_tile_layer(name="World_Imagery",tiles=url, attr ="world imagery" )

    #folium.GeoJson(open("NTSubIbra.json"),style_function=lambda feature: {'color': 'black','weight' : 1,'fillOpacity':0.1}).add_to(m) ## this has commented out as it take a large amount of time to redraw

    #add each site to the web map with its site id. This can a few seconds to add the ~500 sites one by one.
    for i in range(len(df)):  
    
        folium.CircleMarker([df.iloc[i]['lat'], df.iloc[i]['longt']], popup=df.iloc[i]['sites'],radius=50,fill_color='yellow').add_to(m)
            
    display(m)
    

In [5]:
# Click yo display map for site selection
button = widgets.Button(description="click to view map")
display(button)
button.on_click(onMapButtonClick)

#### Enter site details and download image

First four functions are called when the site id is entered and downlaod button clicked (see text box and botton below)

In [6]:
#Function to download selected image and associated list of image dates csv file into local directory

def getDropboxImage(url): 
    
    #image download
    u = urlopen(url)
    data = u.read()
    u.close()
    filename = "chip.tif"
 
    with open(filename, "wb") as f :
        f.write(data)
    
    #pdb.set_trace()
    #dates csv download
    urlTemp = url.replace('images','csvs')
    urlTemp = urlTemp.replace('.tif','.csv')
    u = urlopen(urlTemp)
    data = u.read()
    u.close()
    filename = "dates.csv"
 
    with open(filename, "wb") as f :
        f.write(data)

In [7]:
#Function to match the selected field name with the image name and create the correct url for dropbox download
def matchFileName(siteId):
    
    #loop through  list of images and match to site name 
    for index, row in images.iterrows():
    
        name = row[0]

        nameTemp = name.replace('e','_') # a bit of tidying up to get the obs keys to match

        nameTemp = nameTemp.replace('s','') # a bit of tidying up to get the obs keys to match

        key = nameTemp[7:33] #slice the obskey out of the image name

        dropVal = siteId[:-7] # loose the trailing zeros

        if dropVal == key: #if a match occurs concat to the end of the dropbox public url
            
            #pdb.set_trace()
                       
            url = 'https://dl.dropboxusercontent.com/u/53285700/images/' + name
            
            print url

            return url

    

In [8]:
def doMatch():
    
    mask = df['sites']== text.value
    
    #mask = df['sites']== 'nhv03'
    
    #pdb.set_trace()
    
    if sum(mask) == 0:
        
        print 'no match found - try another site id'
        
    elif sum(mask)>0:
        
        obskey = df['obs_keys'][mask]
    
        siteId = obskey.iloc[0]   
        
        return siteId

In [9]:
#Function that is fired when the download button is clicked
def on_button_clicked(b):
    
    siteId = doMatch()
    
    url = matchFileName(siteId)
   
    print "Downloading please wait...."
    
    getDropboxImage(url)
    
    print "Download Complete"

In [10]:
#on_button_clicked('b')

In [11]:
#Enter siteId from map exactly as it is displayed ie. 'hmd20' or 'bsp14a'
text = widgets.Text(description="site id")
display(text)

In [12]:
# Click download to import geotif image and csv date files to local directory
button = widgets.Button(description="Click to download")
display(button)
button.on_click(on_button_clicked)

https://dl.dropboxusercontent.com/u/53285700/images/lztmre_13353439e02471465s_20121011_dbkm0_e2012fracStack.tif
Download Complete
